# Linear algebra in Julia
Author: Andreas Noack Jensen (http://www.econ.ku.dk/phdstudent/noack/)

## Outline
 - Basic linear algebra 
 - Factorizations (Julia Innovation: Matrix Factorization Objects)
 - Special matrix Structures (Matrices know if they are triangular, tridiagonal, etc.)
 - Generic linear algebra (Increasingly Important and totally outside the scope of LAPACK)

### Basic linear algebra
Syntax very similar to MATLAB but there are some important differences. Define a matrix of random normal variates

In [1]:
A = randn(3,3)

3×3 Array{Float64,2}:
  2.86077   -0.545223  -0.333219
  0.732387  -0.770852   1.50508 
 -1.33949    2.11877   -0.678899

and a vector of ones

In [2]:
x = ones(3)

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

Notice that $A$ has type Array{Float64,2} but $x$ has type Array{Float64,1}. Julia defines the aliases Vector{Type}=Array{Type,1} and Matrix{Type}=Array{Type,2} in Julia. 

Many of the basic operations are the same as in MATLAB
#### Multiplication

In [3]:
b = A*x

3-element Array{Float64,1}:
 1.98232 
 1.46661 
 0.100379

#### Transposition
As in MATLAB `A'` is the conjugate transpose whereas `A.'` is just the transpose

In [4]:
Asym = A + A'

3×3 Array{Float64,2}:
  5.72153    0.187164  -1.67271
  0.187164  -1.5417     3.62384
 -1.67271    3.62384   -1.3578 

#### Transposed multiplication
Julia allows us to write this without *

In [5]:
Apd = A'A

3×3 Array{Float64,2}:
 10.5146   -4.96239   1.05842
 -4.96239   5.38066  -2.41694
  1.05842  -2.41694   2.83719

#### Solving linear systems 
The problem $Ax=b$ for square $A$ is solved by the \ function.

In [6]:
A\b

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

#### Least squares
When A is tall the \ function calculates the least squares solution

In [7]:
A[:,1:2]\b

2-element Array{Float64,1}:
 0.802855
 0.36899 

The \ function also works for rank deficient least squares problems. In this case, the least squares solution is not unique and Julia returns the solution with smallest norm

In [8]:
A = randn(3,3)

3×3 Array{Float64,2}:
 -1.06709   2.28872     0.93659 
  0.136873  1.31067    -0.493879
  0.663735  0.0118471  -1.10897 

In [9]:
[A[:,1] A[:,1]]\b

2-element Array{Float64,1}:
 -0.578223
 -0.578223

#### Underdetermined systems
Minimum norm solution is returned

In [10]:
A[1:2,:]\b[1:2]

3-element Array{Float64,1}:
  0.0416383
  1.00481  
 -0.291444 

### Factorization
The `\` function hides how the problem is actually solved. Depending on the dimensions of `A`, different methods are chosen to solve the problem. An intermediate step in the solution is to calculate a factorization of the matrix `A`. Basically, a factorization of `A` is a way of expressing `A` as a product of triangular, unitary and permutation matrices. Julia defines a `Factorization` abstract type and several composite subtypes for actually storing factorizations. A `Factorization` object should be thought of as a representation of the matrix `A`.

#### LU

When `A` is square, problem is solved by factorizing the matrix `A=PLU` where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular. Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [11]:
A = randn(3,3)

3×3 Array{Float64,2}:
 -0.36687    0.369384  -0.0670999
  0.566249   0.402403   0.812654 
 -1.66461   -0.39709    0.607827 

In [12]:
l,u,p = lu(A)

(
[1.0 0.0 0.0; 0.220394 1.0 0.0; -0.34017 0.585082 1.0],

[-1.66461 -0.39709 0.607827; 0.0 0.456901 -0.201061; 0.0 0.0 1.13706],

[3,1,2])

In [13]:
norm(l*u - A[p,:])

1.2474732967014772e-16

In [14]:
Alu = lufact(A)

Base.LinAlg.LU{Float64,Array{Float64,2}}([-1.66461 -0.39709 0.607827; 0.220394 0.456901 -0.201061; -0.34017 0.585082 1.13706],[3,3,3],0)

The different parts of the factorization can be extracted by special indexing

In [15]:
Alu[:P]

3×3 Array{Float64,2}:
 0.0  0.0  1.0
 1.0  0.0  0.0
 0.0  1.0  0.0

In [16]:
Alu[:L]

3×3 Array{Float64,2}:
  1.0       0.0       0.0
  0.220394  1.0       0.0
 -0.34017   0.585082  1.0

In [17]:
Alu[:U]

3×3 Array{Float64,2}:
 -1.66461  -0.39709    0.607827
  0.0       0.456901  -0.201061
  0.0       0.0        1.13706 

We can therefore compute the solution of $Ax=b$ from the factorization

In [18]:
Alu[:U]\(Alu[:L]\(Alu[:P]'b))

3-element Array{Float64,1}:
 -0.167134
  0.522975
  2.29682 

However, more importantly the `LU` type allows dispatch and we can solve the system by

In [19]:
Alu\ones(3)

3-element Array{Float64,1}:
 -0.805497
  2.04842 
  0.777478

This could be useful if the same left-hand-side is used for several right-hand-sides. The factorization can also be used for calculating the determinant because $\det(A)=\det(PLU)=\det(P)\det(U)=\pm \prod u_{ii}$ because $U$ is triangular and the sign is determined from $\det(P)$.

In [20]:
det(A)

-0.8648004467425022

In [21]:
Alu = lufact(A)
det(Alu)

-0.8648004467425022

#### QR
When `A` is tall, Julia computes the least squares solution $\hat{x}$ that minimizes $\|Ax-b\|_2$. This can be done by factorizing $A=QR$ where $Q$ is unitary/orthogonal and $R=\left(\begin{smallmatrix}R_0\\0\end{smallmatrix}\right)$ and $R_0$ is upper triangular. With the QR factorization the minimum norm can be expressed
\begin{equation*}
\|Ax-b\|=\|QRx-b\|=\|Q(Rx-Q'b)\|=\|Rx-Q'b\|=\left\|\begin{pmatrix}R_0x-Q_0'b\\Q_1'b\end{pmatrix}\right\|=\|R_0x-Q_0'b\|+\|Q_1'b\|
\end{equation*}
and the problem therefore reduces to solving the square problem $R_0x=Q_0'b$ for $x$.

We can QR factorize the submatrix of the two first columns of $A$ by

In [22]:
Aqr = qrfact(A[:,1:2])

Base.LinAlg.QRCompactWY{Float64,Array{Float64,2}}([1.79615 0.419422; -0.261787 -0.529281; 0.769576 -0.390364],[1.20425 1.17501; 6.92621e-310 1.73553])

`\` has a method for the QR and the least squares problem is therefore solved with

In [23]:
Aqr\b

2-element Array{Float64,1}:
 -1.10131
  4.63151

It should be noted that this is *not* the way `A[:,1:2]\b` is solved. In order to handle rank deficient problems Julia uses a QR factorization with pivoting. Pivoting is enabled with a keyword

In [24]:
Aqrp = qrfact([A[:,1] A[:,1]],Val{true})

Base.LinAlg.QRPivoted{Float64,Array{Float64,2}}([1.79615 1.79615; -0.261787 4.57757e-16; 0.769576 -0.780776],[1.20425,1.24254],[1,2])

Notice that the type is different now. `\` also has a method for `QRPivoted` and the rank deficient problem is therefore computed

In [25]:
Aqrp\b

2-element Array{Float64,1}:
 -0.00990024
 -0.00990024

Another feature of the QR factorizations is the `Q` types for storing the unitary matrices $Q$. They can be extracted from the different from `QR` types by indexing

In [26]:
Aqr[:Q]

3×3 Base.LinAlg.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.204254  -0.859756   -0.468082
  0.315257  -0.51046     0.800027
 -0.926765   0.0158422   0.375308

The matrix has a compact internal representation, so indexing is only meaningful if you know how the factorization stores data.

In [27]:
Aqr[:Q][1]

-0.20425362005318792

Even though the matrix is printed as a $3\times 2$ matrix in the factorization object, in practice it can represent the square version as well. Hence both

In [28]:
Aqr[:Q]*ones(2)

3-element Array{Float64,1}:
 -1.06401 
 -0.195202
 -0.910923

and

In [29]:
Aqr[:Q]*ones(3)

3-element Array{Float64,1}:
 -1.53209 
  0.604825
 -0.535615

works, but not

In [30]:
Aqr[:Q]*ones(4)

LoadError: LoadError: DimensionMismatch("vector must have length either 3 or 2")
while loading In[30], in expression starting on line 1

#### Eigendecompositions and the SVD(s)

The results from eigendecompositions and singular value decompositions are also stored in `Factorization` types. This also includes Hessenberg and Schur factorizations.

The eigendecomposition can be computed

In [31]:
AsymEig = eigfact(Asym)

Base.LinAlg.Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}([-5.23166,1.89546,6.15823],[0.11965 0.253365 0.959943; -0.698736 0.708374 -0.0998742; 0.705303 0.658796 -0.261792])

The values and the vectors can be extracted from the Eigen type by special indexing

In [32]:
AsymEig[:values]

3-element Array{Float64,1}:
 -5.23166
  1.89546
  6.15823

In [33]:
AsymEig[:vectors]

3×3 Array{Float64,2}:
  0.11965   0.253365   0.959943 
 -0.698736  0.708374  -0.0998742
  0.705303  0.658796  -0.261792 

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [34]:
inv(AsymEig)*Asym

3×3 Array{Float64,2}:
  1.0          1.249e-16    -3.33067e-16
 -1.22125e-15  1.0          -4.05231e-15
 -5.55112e-16  1.11022e-15   1.0        

Julia also has an `eig` function which returns a tuple with the values and the vectors

In [35]:
eig(Asym)

([-5.23166,1.89546,6.15823],
[0.11965 0.253365 0.959943; -0.698736 0.708374 -0.0998742; 0.705303 0.658796 -0.261792])

This is only provided for MATLAB compatibility and I do not recomment this version.

The `svdfact` function computes the singular value decomposition

In [36]:
Asvd = svdfact(A[:,1:2])

Base.LinAlg.SVD{Float64,Float64,Array{Float64,2}}([-0.143067 -0.872027; 0.350497 -0.486938; -0.925572 -0.0496037],[1.84872,0.51423],[0.96914 0.246511; 0.246511 -0.96914])

and again `\` has a method for the type enabling least squares by SVD

In [37]:
Asvd\b

2-element Array{Float64,1}:
 -1.10131
  4.63151

In contrast to MATLAB, Julia does not allow dispatch on the number of output arguments and therefore there are special functions for providing values only: `eigvals` and `svdvals`.

### Special matrix Structures
The structure of matrices is very important in linear algebra. This structure can be made explicit in Julia through composite types. Examples are `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`. Specialized methods are written for the special matrix types to take advantage of their structure. Below some examples are shown

In [38]:
Diagonal(diag(A))

3×3 Diagonal{Float64}:
 -0.36687   ⋅         ⋅      
   ⋅       0.402403   ⋅      
   ⋅        ⋅        0.607827

In [41]:
LowerTriangular(tril(A))

3×3 LowerTriangular{Float64,Array{Float64,2}}:
 -0.36687     ⋅         ⋅      
  0.566249   0.402403   ⋅      
 -1.66461   -0.39709   0.607827

In [42]:
Symmetric(Asym)

3×3 Symmetric{Float64,Array{Float64,2}}:
  5.72153    0.187164  -1.67271
  0.187164  -1.5417     3.62384
 -1.67271    3.62384   -1.3578 

In [43]:
SymTridiagonal(diag(Asym),diag(Asym,1))

3×3 SymTridiagonal{Float64}:
 5.72153    0.187164    ⋅     
 0.187164  -1.5417     3.62384
  ⋅         3.62384   -1.3578 

When it is known that a matrix is e.g. triangular or symmetric Julia might be able to solve a problem faster by converting the matrix to a special matrix. For some of the procedures, Julia checks if the input matrix is triangular or symmetric and converts the matrix if such a structure is detected. It should noted that `Symmetric`, `Hermitian` and `Triangular` do not copy the input matrix.

#### Symmetric eigenproblem
Whether or not Julia is able to detect if a matrix is symmetric/Hermitian can have a big influence on how fast an eigenvalue problem is solved. Sometimes it is known that a matrix is symmetric or Hermitian but due to floating point errors this is not detected by the `eigvals` function. In following example `Asym1` and `Asym2` are almost identical, but unless Julia is told that `Asym2` is symmetric, the elapsed time for the computation is very different.

In [44]:
n = 1000;
A = randn(n,n);
Asym1 = A + A';
Asym2 = copy(Asym1); Asym2[1,2] += 5eps();
println("Is Asym1 symmetric? ", issymmetric(Asym1))
println("Is Asym2 symmetric? ", issymmetric(Asym2))

Is Asym1 symmetric? true
Is Asym2 symmetric? false


In [45]:
@time eigvals(Asym1);

  0.362096 seconds (27.70 k allocations: 9.143 MB)


In [46]:
@time eigvals(Asym2);

  2.085587 seconds (39 allocations: 7.926 MB)


In [47]:
@time eigvals(Symmetric(Asym2));

  0.252612 seconds (2.79 k allocations: 8.112 MB, 1.85% gc time)


### A big problem
Using tridiagonal matrices makes it possible to work with potentially very large problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a `Matrix` type.

In [48]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  1.139047 seconds (132.25 k allocations: 211.380 MB, 14.43% gc time)


6.174809495785816

### Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does. For a long time Julia has also had support for multiplicaton of general element types. Hence, when multiplying integer matrices, the output is also an integer matrix

In [49]:
rand(1:10,3,3)*rand(1:10,3,3)

3×3 Array{Int64,2}:
 121  217  167
  91  168  133
  39   83   70

Recently, more generic linear algebra methods has been added and Julia now supports generic `LU` and `QR` factorizations. Generic eigenvalue and SVD methods have been written more recently (some in external packages).

In general, the `LU` factorization can be computed whenever the matrix element type is closed under the operations `+`, `-`, `*` and `\`. Of course the matrix also has to have full rank. The generic `LU` method in Julia applies pivoting and therefore the element type also has to support `<` and `abs`. Hence it is possible to solve systems of equations of e.g. rational numbers which the following examples show.

#### Example 1: Rational linear system of equations
Julia has rational numbers built in. The following example shows how a linear system of equations can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [51]:
Ar = convert(Matrix{Rational{BigInt}}, rand(1:10,3,3))/10

3×3 Array{Rational{BigInt},2}:
 1//10  1//1   7//10
 9//10  3//10  9//10
 3//10  1//1   1//10

In [52]:
x = ones(Int,3)
b = Ar*x

3-element Array{Rational{BigInt},1}:
  9//5 
 21//10
  7//5 

In [53]:
Ar\b

3-element Array{Rational{BigInt},1}:
 1//1
 1//1
 1//1

In [54]:
lufact(Ar)

Base.LinAlg.LU{Rational{BigInt},Array{Rational{BigInt},2}}(Rational{BigInt}[1//10 1//1 7//10; 9//1 -87//10 -27//5; 3//1 20//87 -22//29],[1,2,3],0)

#### Example 2: Rational matrix from eigenstructure
The next example shows how rational matrix arithmetic can be used for calculating a matrix given rational eigenvalues and -vectors. I have found this convenient when giving examples of linear dynamic systems.

In [55]:
λ1,λ2,λ3 = 1//1,1//2,1//4
v1,v2,v3 = [1,0,0],[1,1,0],[1,1,1]
V,Λ = [v1 v2 v3], Diagonal([λ1,λ2,λ3])
A = V*Λ/V

3×3 Array{Rational{Int64},2}:
 1//1  -1//2  -1//4
 0//1   1//2  -1//4
 0//1   0//1   1//4